In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


/kaggle/input/airbnb-mexico-city/calendar.csv
/kaggle/input/airbnb-mexico-city/listings.csv
/kaggle/input/airbnb-mexico-city/neighbourhoods.geojson
/kaggle/input/airbnb-mexico-city/reviews.csv


## Practice of ML with analysis for Airbnb in mexico city

In [2]:
calendar_df = pd.read_csv("/kaggle/input/airbnb-mexico-city/calendar.csv")
listings_df = pd.read_csv("/kaggle/input/airbnb-mexico-city/listings.csv")
reviews_df = pd.read_csv("/kaggle/input/airbnb-mexico-city/reviews.csv")

In [3]:
calendar_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,35797,2022-09-22,f,"$4,001.00","$4,001.00",1.0,7.0
1,35797,2022-09-23,f,"$4,001.00","$4,001.00",1.0,7.0
2,35797,2022-09-24,t,"$4,001.00","$4,001.00",1.0,7.0
3,35797,2022-09-25,t,"$4,001.00","$4,001.00",1.0,7.0
4,35797,2022-09-26,t,"$4,001.00","$4,001.00",1.0,7.0


In [4]:
listings_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,35797,https://www.airbnb.com/rooms/35797,20220922013610,2022-09-22,city scrape,Villa Dante,"Dentro de Villa un estudio de arte con futon, ...","Centro comercial Santa Fe, parque interlomas y...",https://a0.muscache.com/pictures/f395ab78-1185...,153786,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
1,783480,https://www.airbnb.com/rooms/783480,20220922013610,2022-09-22,city scrape,"Area Sta.Fe,IberoABC Hospital Rooms",<b>The space</b><br />Two beautifully furnishe...,The neighborhood is very quiet. There are smal...,https://a0.muscache.com/pictures/12600230/eec8...,4129056,...,4.93,4.70,4.85,NaN,t,2,0,2,0,0.26
2,796477,https://www.airbnb.com/rooms/796477,20220922013610,2022-09-23,city scrape,Classic but Modern Apartment in Condesa,Spacious and comfortable room on historic buil...,Condesa is surrounded by important museums and...,https://a0.muscache.com/pictures/58251607/70e9...,4195771,...,4.91,4.96,4.71,NaN,f,1,0,1,0,1.52
3,44616,https://www.airbnb.com/rooms/44616,20220922013610,2022-09-23,city scrape,CONDESA HAUS B&B,A new concept of hosting in mexico through a b...,NaN,https://a0.muscache.com/pictures/251410/ec75fe...,196253,...,4.75,4.98,4.47,NaN,f,12,3,2,0,0.44
4,817291,https://www.airbnb.com/rooms/817291,20220922013610,2022-09-22,city scrape,Live Mexico City Like a Small Town,"Beautiful and cozy apartment in Mexico City, w...",Please check the guidebook. We have posted man...,https://a0.muscache.com/pictures/556d81f5-5aef...,4017148,...,4.95,4.63,4.78,NaN,f,1,1,0,0,0.51


In [5]:
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,44616,706908,2011-11-09,634733,Lindsay,Forget staying in a hotel. Stay at condesa hau...
1,628081,3055971,2012-12-11,837016,Allard,Laura's place is absolutely amazing. Just the ...
2,44616,2006160,2012-08-16,3087087,Samuel,"Fantastic location, great place, friendly host..."
3,44616,3174954,2012-12-28,3234920,Anna,I would highly recommend Condesa Haus for anyo...
4,44616,3271579,2013-01-04,2199822,Shelley And Pall,Great stay. Thanks. Highly recommend.


### Filtering Useful columns in listings df

##### There exists 22948 data in listin df

In [6]:
print(listings_df.columns.tolist())

['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availabil

In [7]:
nulls = pd.DataFrame(listings_df.isnull().sum(axis = 0))
nulls[nulls[0] > 10]

,0
description,1027
neighborhood_overview,8416
host_location,4249
host_about,9999
host_response_time,2945
host_response_rate,2945
host_acceptance_rate,2465
host_neighbourhood,11613
neighbourhood,8416
neighbourhood_group_cleansed,22948


In [8]:
listings_df["review_scores_accuracy"]

0         NaN
1        4.93
2        4.70
3        4.57
4        4.95
         ... 
22943     NaN
22944    5.00
22945    4.85
22946    4.54
22947    4.70
Name: review_scores_accuracy, Length: 22948, dtype: float64

In [9]:
def deleting_columns(x):
    data = x.drop(['listing_url','name','source','host_name','host_thumbnail_url','host_picture_url','neighborhood_overview',
                   'description','picture_url','host_url','host_location','host_about','host_neighbourhood','review_scores_rating','review_scores_accuracy',
                   'review_scores_checkin','review_scores_value','neighbourhood_group_cleansed','neighbourhood','bathrooms','calendar_updated',
                   'license','calendar_last_scraped'],axis=1)
    return data

In [10]:
listings_df = deleting_columns(listings_df)
listings_df

,id,scrape_id,last_scraped,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,last_review,review_scores_cleanliness,review_scores_communication,review_scores_location,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,35797,20220922013610,2022-09-22,153786,2010-06-28,within a day,50%,NaN,f,1,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
1,783480,20220922013610,2022-09-22,4129056,2012-11-12,within a few hours,100%,100%,f,2,...,2019-11-30,4.93,4.93,4.70,t,2,0,2,0,0.26
2,796477,20220922013610,2022-09-23,4195771,2012-11-19,NaN,NaN,67%,t,1,...,2022-05-18,4.70,4.91,4.96,f,1,0,1,0,1.52
3,44616,20220922013610,2022-09-23,196253,2010-08-09,within a few hours,100%,79%,t,16,...,2022-08-29,4.70,4.75,4.98,f,12,3,2,0,0.44
4,817291,20220922013610,2022-09-22,4017148,2012-10-30,within an hour,100%,100%,f,2,...,2021-10-31,4.80,4.95,4.63,f,1,1,0,0,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22943,697457011724147393,20220922013610,2022-09-22,226551717,2018-11-19,NaN,NaN,NaN,f,1,...,NaN,NaN,NaN,NaN,f,1,0,0,1,NaN
22944,22175774,20220922013610,2022-09-22,46746429,2015-10-16,NaN,NaN,NaN,f,1,...,2019-08-17,5.00,5.00,5.00,f,1,0,1,0,0.06
22945,1911707,20220922013610,2022-09-22,641535,2011-05-30,within a few hours,100%,100%,t,1,...,2022-08-24,4.96,4.93,4.81,f,1,1,0,0,0.27
22946,45285491,20220922013610,2022-09-22,365965170,2020-09-04,within an hour,100%,99%,f,8,...,2022-09-11,4.64,4.29,4.82,t,8,6,2,0,2.90


In [11]:
listings_df['price']

0         $4,001.00
1           $500.00
2           $599.00
3        $18,000.00
4           $740.00
            ...    
22943       $450.00
22944       $750.00
22945     $5,041.00
22946     $1,100.00
22947       $959.00
Name: price, Length: 22948, dtype: object

In [12]:
listings_df.columns

Index(['id', 'scrape_id', 'last_scraped', 'host_id', 'host_since',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'first_review', 'last_review',
       'review_scores_cleanliness', 'review_scores_communication',
       'review_score

In [13]:
dea = pd.merge(listings_df, calendar_df, left_on="id",right_on='listing_id')
dea

,id,scrape_id,last_scraped,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,listing_id,date,available,price_y,adjusted_price,minimum_nights_y,maximum_nights_y
0,35797,20220922013610,2022-09-22,153786,2010-06-28,within a day,50%,NaN,f,1,...,0,0,NaN,35797,2022-09-22,f,"$4,001.00","$4,001.00",1.0,7.0
1,35797,20220922013610,2022-09-22,153786,2010-06-28,within a day,50%,NaN,f,1,...,0,0,NaN,35797,2022-09-23,f,"$4,001.00","$4,001.00",1.0,7.0
2,35797,20220922013610,2022-09-22,153786,2010-06-28,within a day,50%,NaN,f,1,...,0,0,NaN,35797,2022-09-24,t,"$4,001.00","$4,001.00",1.0,7.0
3,35797,20220922013610,2022-09-22,153786,2010-06-28,within a day,50%,NaN,f,1,...,0,0,NaN,35797,2022-09-25,t,"$4,001.00","$4,001.00",1.0,7.0
4,35797,20220922013610,2022-09-22,153786,2010-06-28,within a day,50%,NaN,f,1,...,0,0,NaN,35797,2022-09-26,t,"$4,001.00","$4,001.00",1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8375645,45318880,20220922013610,2022-09-22,365965170,2020-09-04,within an hour,100%,99%,f,8,...,2,0,1.62,45318880,2023-09-17,t,$960.00,$960.00,1.0,1125.0
8375646,45318880,20220922013610,2022-09-22,365965170,2020-09-04,within an hour,100%,99%,f,8,...,2,0,1.62,45318880,2023-09-18,t,$960.00,$960.00,1.0,1125.0
8375647,45318880,20220922013610,2022-09-22,365965170,2020-09-04,within an hour,100%,99%,f,8,...,2,0,1.62,45318880,2023-09-19,t,$960.00,$960.00,1.0,1125.0
8375648,45318880,20220922013610,2022-09-22,365965170,2020-09-04,within an hour,100%,99%,f,8,...,2,0,1.62,45318880,2023-09-20,t,$960.00,$960.00,1.0,1125.0


In [14]:
dea.to_csv('Merged_listings.csv')

In [15]:
dea.to_pickle('listings_abnb.csv')